In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/BR_audience_furniture_browser'

In [2]:
df_test=pd.read_csv("./BL_furniture_browser_test_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)
df_control=pd.read_csv("./BL_furniture_browser_control_group_JL_+str(datetime.datetime.now().date()).csv",dtype=str)

print(df_test.shape,df_control.shape)

df_test.head(2)

(489861, 5) (54430, 5)


,index,customersummary_mastercustomerid,Email,customer_id_hashed,execution_id
0,0,KFK_0_8017815,ad5f11c041c89839c09204f9ce34f5d97ea8ddaa81f1f0...,83228bfe2f6d62bb0218751ceb63aa8551e9fd89a999f1...,1578944138509
1,2,KFK_0_37564566,c612188e441b8fef2e5eed0c918fec1d12e23f801b9e69...,94245ca46539f480c2490a5e3373f237dbc455094f85b7...,1578944138509


In [3]:
df_group_1=df_test[['customer_id_hashed']].drop_duplicates()
df_group_1['group']='test'
df_group_2=df_control[['customer_id_hashed']].drop_duplicates()
df_group_2['group']='control'

df_group=df_group_1.append(df_group_2)
print(df_group.shape,df_group['customer_id_hashed'].nunique())

df_total_id=df_group.groupby(['group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"total_ids"})

(544291, 2) 544291


In [4]:
df_total_id

,group,total_ids
0,control,54430
1,test,489861


In [5]:
df_sales_last_week=pd.read_table("/home/jian/BigLots/MediaStorm_2020-01-18/MediaStormDailySales20200121-111749-649.txt",
                                sep="|",dtype=str)
df_sales_last_week.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_id,item_transaction_units,item_transaction_amt
0,1027,2020-01-18,2108,51cc9db6c31bf222baea4e50b9a6b1be2c7c4ec833bcfe...,14001,1,140008972,1,00000000000001.6000
1,1960,2020-01-14,9803,NaN,55006,1,810330818,-1,-00000000000002.5000


In [6]:
df_sales_last_week['item_transaction_amt']=df_sales_last_week['item_transaction_amt'].astype(float)
df_sales_last_week=df_sales_last_week[pd.notnull(df_sales_last_week['customer_id_hashed'])]
df_sales_last_week=pd.merge(df_sales_last_week,df_group,on="customer_id_hashed",how="left")
df_sales_last_week['group']=df_sales_last_week['group'].fillna('others')

In [7]:
df_taxonomy_div=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",dtype=str,sep="|")
df_taxonomy_div=df_taxonomy_div[['division_id','class_code_id','subclass_id']].drop_duplicates()

df_sales_last_week=pd.merge(df_sales_last_week,df_taxonomy_div,on=['class_code_id','subclass_id'],how="left")
df_sales_last_week['store_type']=np.where(df_sales_last_week['location_id']=="6990","online","instore")

# No Store Type

In [8]:
agg_fun={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
df_total_sales=df_sales_last_week.groupby(['transaction_dt','group'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_last_week[['group','location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_last_week.groupby(['transaction_dt','group','division_id'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_last_week[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','division_id'])['trans'].sum().to_frame().reset_index()


In [9]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','division_id'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [10]:
pd.options.display.max_columns=99

df_output=pd.merge(df_total,df_division,on=['transaction_dt','group'],how="outer")

df_output=pd.merge(df_total_id,df_output,on="group",how="outer")
df_output=df_output.sort_values(["transaction_dt","group"])
df_output

,group,total_ids,transaction_dt,total_shoppers,total_sales,total_trans,total_AOV,division_1_shoppers,division_1_sales,division_1_trans,division_1_AOV,division_2_shoppers,division_2_sales,division_2_trans,division_2_AOV,division_3_shoppers,division_3_sales,division_3_trans,division_3_AOV,division_4_shoppers,division_4_sales,division_4_trans,division_4_AOV,division_5_shoppers,division_5_sales,division_5_trans,division_5_AOV,division_6_shoppers,division_6_sales,division_6_trans,division_6_AOV,division_9_shoppers,division_9_sales,division_9_trans,division_9_AOV
0,control,54430.0,2020-01-12,794,5.988540e+04,906,66.098675,462,4.418580e+03,482,9.167178,392,5991.81,409,14.649902,153,2404.16,161,14.932671,327,9.485220e+03,350,27.100629,192,3214.92,197,16.319391,165,1619.49,170,9.526412,140,3.274867e+04,145,225.852897
14,others,NaN,2020-01-12,177861,7.398193e+06,198934,37.189182,111186,1.158330e+06,118286,9.792622,101041,1378900.83,106475,12.950466,31252,440708.90,32588,13.523656,61038,1.419223e+06,64418,22.031463,38503,497544.88,39793,12.503327,33113,373372.79,34656,10.773684,8690,2.127883e+06,9288,229.100277
7,test,489861.0,2020-01-12,7148,6.245568e+05,8322,75.048885,4070,4.272876e+04,4357,9.806922,3572,49885.18,3760,13.267335,1403,27790.87,1469,18.918223,3101,8.523888e+04,3309,25.759710,1685,24602.21,1770,13.899554,1268,15558.29,1326,11.733250,1248,3.786506e+05,1364,277.603057
1,control,54430.0,2020-01-13,585,3.291648e+04,670,49.129075,316,2.972000e+03,332,8.951807,259,2839.44,263,10.796350,145,2703.99,162,16.691296,236,6.255900e+03,251,24.923904,99,983.31,104,9.454904,88,668.13,91,7.342088,78,1.649097e+04,88,187.397386
15,others,NaN,2020-01-13,152981,4.895059e+06,172919,28.308392,97264,9.658510e+05,104263,9.263603,80305,993020.83,84488,11.753395,36983,437322.35,40969,10.674470,49919,9.495068e+05,53359,17.794688,30841,339000.84,32024,10.585837,26351,253393.19,27524,9.206263,4478,9.553673e+05,4712,202.751984
8,test,489861.0,2020-01-13,5158,2.867841e+05,6022,47.622732,2825,2.710999e+04,3024,8.964944,2300,27748.34,2404,11.542571,1402,21838.00,1581,13.812777,2099,4.695510e+04,2281,20.585313,1078,13597.65,1126,12.076066,796,8303.13,832,9.979724,647,1.411771e+05,723,195.265629
2,control,54430.0,2020-01-14,520,2.349924e+04,603,38.970547,304,2.924410e+03,317,9.225268,247,3234.04,252,12.833492,133,1660.74,145,11.453379,176,3.597720e+03,195,18.449846,102,922.31,105,8.783905,84,583.05,87,6.701724,44,1.057492e+04,51,207.351373
16,others,NaN,2020-01-14,151259,4.671873e+06,169988,27.483549,97695,9.666806e+05,104651,9.237184,79429,977231.19,83616,11.687132,34130,350136.31,36999,9.463399,46696,8.987972e+05,49544,18.141394,29492,322582.58,30579,10.549154,25813,257475.95,26913,9.566973,4191,8.972139e+05,4430,202.531359
9,test,489861.0,2020-01-14,4757,2.521696e+05,5544,45.485133,2864,2.906445e+04,3067,9.476508,2349,29625.39,2472,11.984381,1231,17574.64,1318,13.334325,1795,3.735968e+04,1927,19.387483,1003,9702.22,1050,9.240210,772,6943.93,801,8.669076,485,1.218368e+05,539,226.042208
3,control,54430.0,2020-01-15,543,2.976372e+04,619,48.083554,307,3.097720e+03,326,9.502209,266,3154.22,275,11.469891,109,2939.90,121,24.296694,205,4.418710e+03,225,19.638711,102,1347.78,107,12.596075,75,931.48,80,11.643500,48,1.387141e+04,52,266.757885


# By Store Type Online/Instore

In [11]:
agg_fun={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
df_total_sales=df_sales_last_week.groupby(['transaction_dt','group','store_type'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_total_trans=df_sales_last_week[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','store_type']].drop_duplicates()
df_total_trans['trans']=1
df_total_trans=df_total_trans.groupby(['transaction_dt','group','store_type'])['trans'].sum().to_frame().reset_index()
df_total=pd.merge(df_total_sales,df_total_trans,on=['transaction_dt','group','store_type'],how="outer")
df_total=df_total.rename(columns={"customer_id_hashed":"total_shoppers","item_transaction_amt":"total_sales","trans":"total_trans"})
df_total['total_AOV']=df_total['total_sales']/df_total['total_trans']

df_division_sales=df_sales_last_week.groupby(['transaction_dt','group','division_id','store_type'])['customer_id_hashed','item_transaction_amt'].agg(agg_fun).reset_index()
df_division_trans=df_sales_last_week[['group','location_id','transaction_dt','transaction_id','customer_id_hashed','division_id','store_type']].drop_duplicates()
df_division_trans['trans']=1
df_division_trans=df_division_trans.groupby(['transaction_dt','group','division_id','store_type'])['trans'].sum().to_frame().reset_index()


In [12]:
df_division=pd.merge(df_division_sales,df_division_trans,on=['transaction_dt','group','division_id','store_type'],how="outer")
df_division['AOV']=df_division['item_transaction_amt']/df_division['trans']
df_division=df_division.rename(columns={"customer_id_hashed":"division_shoppers","item_transaction_amt":"division_sales","trans":"division_trans","AOV":"division_AOV"})

df_division=df_division.pivot_table(index=['transaction_dt','group','store_type'],columns='division_id',values=['division_shoppers','division_sales','division_trans',"division_AOV"]).reset_index()

new_cols=[]
for col in df_division.columns.tolist():
    if not col[1]:
        new_cols.append(col[0])
    else:
        new_cols.append("division_"+col[1]+"_"+col[0].split("_")[1])
df_division.columns=new_cols

new_cols=['transaction_dt','group','store_type']
for i in range(10):
    new_cols.append("division_"+str(i)+"_shoppers")
    new_cols.append("division_"+str(i)+"_sales")
    new_cols.append("division_"+str(i)+"_trans")
    new_cols.append("division_"+str(i)+"_AOV")
new_cols=[x for x in new_cols if x in df_division.columns.tolist()]    
df_division=df_division[new_cols]

In [13]:
pd.options.display.max_columns=99

df_output_2=pd.merge(df_total,df_division,on=['transaction_dt','group','store_type'],how="outer")

df_output_2=pd.merge(df_total_id,df_output_2,on="group",how="outer")
df_output_2=df_output_2.sort_values(["transaction_dt","group",'store_type'])
df_output_2

,group,total_ids,transaction_dt,store_type,total_shoppers,total_sales,total_trans,total_AOV,division_1_shoppers,division_1_sales,division_1_trans,division_1_AOV,division_2_shoppers,division_2_sales,division_2_trans,division_2_AOV,division_3_shoppers,division_3_sales,division_3_trans,division_3_AOV,division_4_shoppers,division_4_sales,division_4_trans,division_4_AOV,division_5_shoppers,division_5_sales,division_5_trans,division_5_AOV,division_6_shoppers,division_6_sales,division_6_trans,division_6_AOV,division_9_shoppers,division_9_sales,division_9_trans,division_9_AOV
0,control,54430.0,2020-01-12,instore,773,5.867279e+04,883,66.447101,462.0,4.418580e+03,482.0,9.167178,392.0,5991.81,409.0,14.649902,152.0,2382.76,160.0,14.892250,317.0,9.037240e+03,339.0,26.658525,189.0,3134.03,193.0,16.238497,161.0,1517.82,166.0,9.143494,132.0,3.218800e+04,136.0,236.676471
1,control,54430.0,2020-01-12,online,21,1.212610e+03,23,52.722174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,21.40,1.0,21.400000,10.0,4.479800e+02,11.0,40.725455,3.0,80.89,4.0,20.222500,4.0,101.67,4.0,25.417500,8.0,5.606700e+02,9.0,62.296667
28,others,NaN,2020-01-12,instore,177480,7.378590e+06,198515,37.168928,111163.0,1.156803e+06,118263.0,9.781614,101036.0,1378848.27,106470.0,12.950580,31171.0,438568.31,32500.0,13.494410,60823.0,1.411178e+06,64191.0,21.984042,38466.0,495955.54,39756.0,12.474986,33022.0,368832.39,34556.0,10.673469,8668.0,2.126176e+06,9263.0,229.534243
29,others,NaN,2020-01-12,online,399,1.960284e+04,419,46.784821,23.0,1.527120e+03,23.0,66.396522,5.0,52.56,5.0,10.512000,83.0,2140.59,88.0,24.324886,222.0,8.045150e+03,227.0,35.441189,37.0,1589.34,37.0,42.955135,95.0,4540.40,100.0,45.404000,23.0,1.707680e+03,25.0,68.307200
14,test,489861.0,2020-01-12,instore,6955,6.113767e+05,8100,75.478604,4067.0,4.256190e+04,4354.0,9.775356,3570.0,49872.28,3758.0,13.270963,1353.0,26549.14,1418.0,18.722948,3005.0,8.213670e+04,3207.0,25.611693,1668.0,24230.60,1753.0,13.822362,1223.0,13996.88,1281.0,10.926526,1197.0,3.719271e+05,1301.0,285.877886
15,test,489861.0,2020-01-12,online,204,1.318013e+04,222,59.369955,3.0,1.668600e+02,3.0,55.620000,2.0,12.90,2.0,6.450000,51.0,1241.73,51.0,24.347647,100.0,3.102180e+03,102.0,30.413529,17.0,371.61,17.0,21.859412,45.0,1561.41,45.0,34.698000,52.0,6.723440e+03,63.0,106.721270
2,control,54430.0,2020-01-13,instore,528,2.440962e+04,593,41.162934,314.0,2.930890e+03,330.0,8.881485,259.0,2839.44,263.0,10.796350,136.0,2088.43,153.0,13.649869,211.0,4.714910e+03,220.0,21.431409,96.0,929.17,101.0,9.199703,86.0,647.43,89.0,7.274494,48.0,1.025661e+04,48.0,213.679375
3,control,54430.0,2020-01-13,online,60,8.506860e+03,77,110.478701,2.0,4.111000e+01,2.0,20.555000,NaN,NaN,NaN,NaN,9.0,615.56,9.0,68.395556,27.0,1.540990e+03,31.0,49.709355,3.0,54.14,3.0,18.046667,2.0,20.70,2.0,10.350000,31.0,6.234360e+03,40.0,155.859000
30,others,NaN,2020-01-13,instore,152346,4.837609e+06,172129,28.104553,97229.0,9.631934e+05,104228.0,9.241216,80297.0,992709.30,84479.0,11.750959,36849.0,427449.32,40816.0,10.472592,49585.0,9.331797e+05,52974.0,17.615806,30790.0,336509.82,31967.0,10.526788,26272.0,250716.96,27436.0,9.138248,4356.0,9.322535e+05,4554.0,204.710905
31,others,NaN,2020-01-13,online,669,5.745030e+04,790,72.721899,35.0,2.657540e+03,35.0,75.929714,9.0,311.53,9.0,34.614444,140.0,9873.03,153.0,64.529608,344.0,1.632706e+04,385.0,42.407948,55.0,2491.02,57.0,43.702105,82.0,2676.23,88.0,30.411705,123.0,2.311389e+04,158.0,146.290443


In [14]:
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")

In [15]:
writer=pd.ExcelWriter("./BL_performance_of_furniture_browser_groups_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output.to_excel(writer,"view_no_store_type",index=False)
df_output_2.to_excel(writer,"view_with_store_type",index=False)
df_division_name.to_excel(writer,"divison_name",index=False)
writer.save()

In [17]:
sorted(df_sales_last_week['division_id'].unique().tolist())

['1', '10', '2', '3', '4', '5', '6', '9']